## Exploring the Data

In [1]:
import os
import polars as pl
from utils import *
import pandas as pd

Let's read the data corresponding to a single state ('AC' in this case) to explore it.

In [2]:
csv_files_dir = '../data/csv_files'
file = 'SP.csv'
file_path = os.path.join(csv_files_dir, file)
df = read_csv(file_path, separator = ',')

Let's check the different types of 'ClassInfraFisica' present.

In [ ]:
df.group_by('ClassInfraFisica').count()

Let's fix the typo:

In [3]:
df = strip_column(df, 'ClassInfraFisica')

In [4]:
df.group_by('ClassInfraFisica').agg(pl.col('ClassInfraFisica').count().alias('Count'))

ClassInfraFisica,Count
str,u32
null,2067448
"""Outdoor""",143
"""Harmonizada""",288
"""Fastsite""",87
"""COW""",1339
"""Greenfield""",165328
"""Streetlevel""",812
"""Rooftop""",49716
"""Ran Sharing""",2143


Now, we have to check whether each station (indicated by the column 'NumEstacao') corresponds to a single type of 'ClassInfraFisica'; which is the expected behavior (assumption).

In [5]:
count_unique_values = df.group_by('NumEstacao').agg(pl.col('ClassInfraFisica').n_unique().alias('unique_count'))
instances_with_different_values = count_unique_values.filter(count_unique_values['unique_count'] > 1)
print(instances_with_different_values)

shape: (8_081, 2)
┌────────────┬──────────────┐
│ NumEstacao ┆ unique_count │
│ ---        ┆ ---          │
│ i64        ┆ u32          │
╞════════════╪══════════════╡
│ 1007447238 ┆ 2            │
│ 687422329  ┆ 2            │
│ 698139917  ┆ 2            │
│ 683889192  ┆ 2            │
│ …          ┆ …            │
│ 1008765098 ┆ 2            │
│ 1011786505 ┆ 2            │
│ 689693885  ┆ 2            │
│ 641263155  ┆ 2            │
└────────────┴──────────────┘


We found that we can actually get two different 'ClassInfraFisica' for any individual station. But if we check further, we'll see that it's actually pairing types with 'null' values, which does not compromise the assumption.

In [6]:
df.filter(df['NumEstacao'] == 1007447238)[['NumEstacao', 'ClassInfraFisica']]

NumEstacao,ClassInfraFisica
i64,str
1007447238,"""Greenfield"""
1007447238,"""Greenfield"""
1007447238,"""Greenfield"""
1007447238,"""Greenfield"""
1007447238,"""Greenfield"""
1007447238,"""Greenfield"""
1007447238,"""Greenfield"""
1007447238,"""Greenfield"""
1007447238,"""Greenfield"""


Let's group by 'NumEstacao', but keeping all the info present in the rows as sets (or as a single value if all rows match)

In [ ]:
dfp = df.to_pandas()          
def set_aggregation(x):

    if len(set(x)) == 1:
        return x.iloc[0]  
    else:
        return set(x)  

dfg = dfp.groupby('NumEstacao').agg(lambda x: set_aggregation(x))
dfg.head(4)

In [ ]:
print(f"Percentage of initial rows kept when grouping = {len(dfg)*100/len(df):.2f}%")

Now we have a dataframe with all the original information, but only 16% of the rows; in which each represents a single station

## Understanding the Columns

Let's understand what the columns are about. This will futurely enable us to better make sense of them, process them if necessary, or drop irrelevant ones.

In [41]:
pandas_df = df.to_pandas()

pandas_df['Technology'] = pandas_df.apply(lambda row: concatenate_columns(row, 'Tecnologia', 'tipoTecnologia'), axis=1)



,tech
12419,NR_SA-NSA
12424,NR_SA-NSA
12425,NR_SA-NSA
13863,NR_SA-NSA
13864,NR_SA-NSA
...,...
2293519,NR_NSA
2293520,NR_NSA
2293521,NR_NSA
2293522,NR_NSA


From [Página de Emissões - Anatel](https://sistemas.anatel.gov.br/anexar-api/publico/anexos/download/9cfc11fc83fcfc2d2586cdb887f72cb5), we can understand what each column refers to.

Status.state: Situação da solicitação na Anatel.

NumFistel: É um código numérico definido pela Anatel, composto de onze dígitos que identifica a autorização que a empresa possui para determinado serviço em determinada região.

NumServico: Código do serviço de telecomunicações na Anatel.

NumAto: Número do Ato de Autorização de Uso de Radiofrequência, sendo que os quatro últimos dígitos se referem ao ano da assinatura do respectivo ato. Exemplo XYZW2017, ato XYZW de 2017.

EnderecoEstacao: Endereço completo onde a estação está instalada.

EndComplemento: Complemento do endereço, caso haja.

DesignacaoEmissao: Baseado em [Método de Designação - Anatel](https://www.anatel.gov.br/Portal/verificaDocumentos/documento.asp?numeroPublicacao=60403&assuntoPublicacao=null&caminhoRel=null&filtro=1&documentoPath=outros/autocadastramento/metodo_de_designacao.pdf)

Designação de emissão. Para uma completa designação da emissão, necessitamos sempre de 9 caracteres alfanuméricos. Os quatros primeiros representam a _largura de faixa necessária_; os três seguintes as _características básicas_ e os dois últimos as _características adicionais facultativas_.
- Largura de faixa necessária: Para uma dada classe de emissão, o valor mínimo da largura de faixa ocupada pela emissão, suficiente para garantir a transmissão da informação com a velocidade de transmissão e com a qualidade requerida para o sistema empregado, nas condições especificadas. A largura de faixa necessária será sempre expressa por meio de três algarismos, que indicam os três primeiros algarismos significativos de largura de faixa necessária e uma letra que ocupa a posição da virgula decimal e representa a Unidade de largura de faixa e será H para Hertz, K para Kilohertz, M para Megahertz ou G para Gigahertz.
- Características básicas: descritas por três símbolos:
    - Primeiro símbolo: tipo de modulação da portaria principal.
    - Segundo símbolo: natureza do(s) sinais que modulam a portaria principal.
    - Terceiro símbolo: tipo de informação a ser transmitida.
- Características adicionais facultativas: Para descrição mais completa de uma emissão, são previstas duas características facultativas, as quais são expressas pelos quarto e quinto símbolos. Quando não se utiliza o quarto ou o quinto símbolo, convém indicar isso mediante um traço no lugar em que cada símbolo apareceria.

meioAcesso: é um campo de ‘estações dispensadas de licenciamento’. As opções são: fibra, par metálico, cabo coaxial e radiação restrita.

Azimute: posicionamento m graus em relação ao Norte do lóbulo principal de radiação da antena. Quando for utilizada antena omnidirecional será 0.

CodTipoClasseEstacao: Classe da Estação conforme lista permitida no sistema de canalização do Mosaico, conforme o Regulamento da faixa de frequência utilizada conforme Manual de Projetos Técnicos do SITAR, disponível em [Manual de Projeto - Anatel](https://www.anatel.gov.br/Portal/verificaDocumentos/documento.asp?numeroPublicacao=60402&assuntoPublicacao=MANUAL%20DE%20PROJETOS%20T%C9CNICOS%20(SITAR)&caminhoRel=CidadaoComunica%E7%E3o%20via%20R%E1dioServi%E7o%20Limitado&filtro=1&documentoPath=outros/). Exemplos:
- ML: Estação móvel terrestre
- FX: Estação fixa
- FB: Estação de base
- XR: Estação fixa repetidora
- FA: Estação aeronáutica
- RC: Radiofarol não direcional

ClassInfraFisica: Classificação de Infraestrutura Física. Apenas para estações dos serviços de interesse coletivo (STFC, SCM e SMP). Para serviços de interesse restrito (SLP), estará em branco. Valores válidos: Greenfield, Streetlevel, Ran Sharing, Small Cell, COW, Picocelula, Harmonizada, Rooftop, Indoor, Outdoor

CompartilhamentoInfraFisica: Compartilhamento de Infraestrutura Física. Apenas para estações dos serviços de interesse coletivo (não se aplica para SLP).

CodTipoAntena: Tipo Antena: campo obrigatório, onde é inserido o código conforme item 2.3.4 do [Manual de Projeto - Anatel](https://www.anatel.gov.br/Portal/verificaDocumentos/documento.asp?numeroPublicacao=60402&assuntoPublicacao=MANUAL%20DE%20PROJETOS%20T%C9CNICOS%20(SITAR)&caminhoRel=CidadaoComunica%E7%E3o%20via%20R%E1dioServi%E7o%20Limitado&filtro=1&documentoPath=outros/). Exemplos:
- 019: Monopolo vertical
- 060: V invertido
- 213: Cabo fendido

GanhoAntena: Ganho da Antena em dB. Se a frequência for superior a 28.000 kHz, é informado o ganho da antena em relação a uma antena isotrópica (dbi); se inferior, este é dado em relação a uma antena dipolo (dBd).

FrenteCostaAntena: Relação Frente/Costa em dBi (campo obrigatório). Deve ser menor que 90 dbi. Será 0 (zero) quando for utilizada antena omnidirecional.

AnguloMeiaPotenciaAntena: Ângulo de meia potência em graus decimais (campo obrigatório). Pontos no diagrama onde a potência radiada equivale à metade da radiada na direção principal.

AnguloElevacao: : Ângulo de elevação mecânico em graus decimais. O valor deve estar entre 0 e 90 e ser medido com relação à linha de horizonte, sendo negativo quando a linha de visada estiver abaixo desta referência.

Polarizacao:  a orientação do campo elétrico da onda de rádio com respeito a terra ou direção de propagação; e é determinado pela estrutura física da antena e por sua orientação. Valores aceitos: H (horizontal), V (vertical), CR (circular á direita), CL (circular à esquerda) e X (horizontal e vertical simultâneas ou não).

AlturaAntena: Altura da antena (em relação a cota do terreno). Deve ser menor que 200 m.

PotenciaTransmissorWatts: Potência nominal (saída do transmissor) em watts.

DataLicenciamento:  Data do último licenciamento da estação. Formato: AAAA/MM/DD

DataPrimeiroLicenciamento: Data do primeiro licenciamento da estação. Formato: AAAA/MM/DD

NumRede: Valor numérico, definido pelo usuário, para identificar grupo de estações que se comunicam. Visa facilitar a análise da comunicação das redes.

_id:  Identificação da emissão / frequência da estação

DataValidade: Validade da RF associada àquela estação.

NumFistelAssociado: Campo preenchido quando a emissão for de outro Fistel (do mesmo CNPJ ou diferentes): licenciamento conjunto. Nesse caso, é informado o número do Fistel responsável pelo Ato de RF da emissão dessa linha



meioAcesso consists of all None values

In [ ]:
# meioAcesso
dfg['meioAcesso'].value_counts(dropna=False)

In [ ]:
dfg['CodTipoClasseEstacao'].value_counts()